In [4]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [5]:
import imageio  # make sure to install imageio with mamba install imageio
import pims_nd2
import nd2_to_caiman
import h5py
import numpy as np

In [6]:
nd2_path = "D:/PhD/Data/T386_MatlabTest/T386_20211202_green.nd2"
export_file = nd2_path[:-4] + "_exp.tiff"

## Get data as np array

In [9]:
nd2_file = pims_nd2.ND2_Reader(nd2_path)

# might need to save the metadata too
nd2_metadata = nd2_file.metadata

In [10]:
export_arr = nd2_to_caiman.np_arr_from_nd2(nd2_path)  # FIXME: reading out results in same frames
export_arr.shape

(577, 512, 512)

# TIFF with imageio: probably not multi-page.

In [18]:
imageio.mimwrite(export_file, export_arr)

ValueError: Could not find a backend to open `D:/PhD/Data/T386_MatlabTest/T386_20211202_green_exp.h5`` with iomode `wI`.

In [6]:
# check if images are same
export_arr[0] == export_arr[100]

array([[False, False, False, ..., False,  True,  True],
       [ True,  True, False, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

# HDF5 with h5py

In [11]:
h5_file = export_file[:-5] + ".h5"
print(h5_file)

D:/PhD/Data/T386_MatlabTest/T386_20211202_green_exp.h5


In [12]:
dataset_name = "mov"  # an annoying bug in CaImAn... passing 'var_name_hdf5': 'data' 
                      # to CNMFParams does not work. Stick to default name for now
with h5py.File(h5_file, 'w') as hf:
    dataset = hf.create_dataset("mov", shape=export_arr.shape, dtype=np.uint16)
    for i_frame in range(export_arr.shape[0]):
        dataset[i_frame] = export_arr[i_frame]

In [26]:
export_arr.shape[0]

577

In [25]:
export_arr[0]

array([[ 0,  0,  0, ...,  1,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  1,  0, ...,  3,  0,  0],
       ...,
       [15, 23, 31, ..., 19, 29, 12],
       [16, 21, 30, ..., 21, 38, 13],
       [24, 26, 33, ..., 39, 43, 26]], dtype=uint16)

In [5]:
f = h5py.File(h5_file, "r")

In [6]:
f.get("data")

<HDF5 dataset "data": shape (577, 512, 512), type "<u2">